In [1]:
import os, shutil

src = "/kaggle/input/stdc-repo/STDC-Seg"
dst = "/kaggle/working/STDC-Seg"

# ensure the parent exists
os.makedirs(os.path.dirname(dst), exist_ok=True)

# copy the whole tree
shutil.copytree(src, dst)

'/kaggle/working/STDC-Seg'

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os
import sys
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
sys.path.append('/kaggle/working/STDC-Seg')

import argparse
import logging
import time
import datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from loveda import LoveDA
from models.model_stages import BiSeNet
from loss.loss import WeightedOhemCELoss, OhemCELoss
from loss.detail_loss import DetailAggregateLoss
from optimizer_loss import Optimizer
import torch.optim as optim
# Configuration
def parse_args():
    parser = argparse.ArgumentParser(description='STDC Segmentation Training')
    parser.add_argument('--epochs', type=int, default=20, help='number of training epochs')
    parser.add_argument('--batch_size', type=int, default=6, help='images per batch')
    parser.add_argument('--n_workers_train', type=int, default=4)
    parser.add_argument('--n_workers_val', type=int, default=0)
    parser.add_argument('--cropsize', type=int, nargs=2, default=[1024, 512])
    parser.add_argument('--randomscale', type=float, nargs='+',
                        default=[0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 1.125, 1.25, 1.375, 1.5])
    parser.add_argument('--backbone', type=str, default='STDCNet813')
    parser.add_argument('--pretrain_path', type=str, default='/kaggle/working/STDC-Seg/pretrained_models/STDCNet813M_73.91.tar')
    parser.add_argument('--ckpt', type=str, default=None)
    parser.add_argument('--respath', type=str, default='/kaggle/working/output')
    parser.add_argument('--use_conv_last', action='store_true')
    parser.add_argument('--use_boundary_2', action='store_true')
    parser.add_argument('--use_boundary_4', action='store_true')
    parser.add_argument('--use_boundary_8', action='store_true')
    parser.add_argument('--use_boundary_16', action='store_true')
    return parser.parse_args(args=[] if '__file__' not in globals() else None)

# Metrics
def compute_iou(pred, target, num_classes, ignore_index=255):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)
    for cls in range(num_classes):
        mask = target == cls
        if mask.sum().item() == 0:
            ious.append(float('nan'))
            continue
        inter = ((pred == cls) & mask).sum().item()
        union = ((pred == cls) | mask).sum().item()
        ious.append(inter / union if union > 0 else float('nan'))
    miou = np.nanmean(ious)
    return ious, miou

# Training loop
def train():
    args = parse_args()
    os.makedirs(args.respath, exist_ok=True)
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    n_classes = 7
    ignore_idx = 255

    # Dataset and DataLoader
    ds_train = LoveDA('/kaggle/input/loveda-splits', cropsize=args.cropsize,
                      mode='train', randomscale=tuple(args.randomscale), resolution=(512, 512))
    dl_train = DataLoader(ds_train, batch_size=args.batch_size, shuffle=True,
                          num_workers=args.n_workers_train, pin_memory=True, drop_last=False)
    ds_val = LoveDA('/kaggle/input/loveda-splits', mode='val', randomscale=tuple(args.randomscale), resolution=(512, 512))
    dl_val = DataLoader(ds_val, batch_size=args.batch_size, shuffle=False,
                        num_workers=args.n_workers_val, pin_memory=True, drop_last=False)

    logger.info(f"Train samples: {len(ds_train)}, Val samples: {len(ds_val)}")

    # Model
    net = BiSeNet(backbone=args.backbone, n_classes=n_classes,
                  pretrain_model=args.pretrain_path,
                  use_boundary_2=args.use_boundary_2,
                  use_boundary_4=args.use_boundary_4,
                  use_boundary_8=args.use_boundary_8,
                  use_boundary_16=args.use_boundary_16,
                  use_conv_last=args.use_conv_last)
    if args.ckpt:
        net.load_state_dict(torch.load(args.ckpt, map_location='cpu'))
    net = net.to(device)
    net = nn.DataParallel(net)

    # Losses and Optimizer
    n_min = args.batch_size * args.cropsize[0] * args.cropsize[1] // 16
    criteria = {
        'p': WeightedOhemCELoss(0.7, n_min, 7, ignore_lb=ignore_idx),
        '16': WeightedOhemCELoss(0.7, n_min, 7, ignore_lb=ignore_idx),
        '32': WeightedOhemCELoss(0.7, n_min, 7, ignore_lb=ignore_idx)
    }
    boundary_loss = DetailAggregateLoss()
    optimizer = Optimizer(model=net.module, loss=boundary_loss,
                          lr0=1e-3, momentum=0.9, wd=5e-4,
                          warmup_steps=1000, warmup_start_lr=1e-5,
                          max_iter=len(dl_train)*args.epochs,
                          power=0.9)

    
    
    

    # Epoch loop
    best_miou = 0.0
    for epoch in range(1, args.epochs + 1):
        # -- Training --
        net.train()
        running_loss = 0.0
        train_bar = tqdm(dl_train, desc=f"Epoch {epoch}/{args.epochs} Training", unit="batch")
        for imgs, lbs in train_bar:
            imgs = imgs.to(device)
            lbs = lbs.squeeze(1).to(device)

            optimizer.zero_grad()
            outputs = net(imgs)
            out, out16, out32 = outputs[:3]

            lp = criteria['p'](out, lbs)
            l16 = criteria['16'](out16, lbs)
            l32 = criteria['32'](out32, lbs)
            lbce, ldice = 0.0, 0.0
            if args.use_boundary_2 or args.use_boundary_4 or args.use_boundary_8 or args.use_boundary_16:
                for det in outputs[3:]:
                    bce, dice = boundary_loss(det, lbs)
                    lbce += bce; ldice += dice

            loss = lp + l16 + l32 + lbce + ldice
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            train_bar.set_postfix(loss=f"{loss.item():.4f}")

        avg_train_loss = running_loss / len(dl_train)
        print(f"Epoch {epoch} Training complete. Avg Loss: {avg_train_loss:.4f}")
        logger.info(f"Epoch [{epoch}/{args.epochs}] Train Loss: {avg_train_loss:.4f}")

        # -- Validation --
        net.eval()
        running_val_loss = 0.0
        intersection = torch.zeros(n_classes, dtype=torch.float64)
        union        = torch.zeros(n_classes, dtype=torch.float64)
        val_bar = tqdm(dl_val, desc=f"Epoch {epoch}/{args.epochs} Validation", unit="batch")
        with torch.no_grad():
            for imgs, lbs in val_bar:
                imgs = imgs.to(device)
                lbs = lbs.squeeze(1).to(device)
                outputs = net(imgs)
                out = outputs[0]

                # validation loss
                lval = criteria['p'](out, lbs)
                running_val_loss += lval.item()
                val_bar.set_postfix(valloss=f"{lval.item():.4f}")

                pred = torch.argmax(out, dim=1)
                for cls in range(n_classes):
                    mask = (lbs == cls)
                    if mask.sum().item() == 0:
                        continue
                    inter = ((pred == cls) & mask).sum().item()
                    uni   = ((pred == cls) | mask).sum().item()
                    intersection[cls] += inter
                    union[cls]        += uni

        avg_val_loss = running_val_loss / len(dl_val)
        print(f"Epoch {epoch} Validation complete. Avg Loss: {avg_val_loss:.4f}")
        logger.info(f"Epoch [{epoch}/{args.epochs}] Val Loss: {avg_val_loss:.4f}")

        ious = (intersection / union).tolist()
        miou = float(torch.nanmean(intersection / union))
        print("Per-class IoU:")
        for cls_idx, cls_iou in enumerate(ious):
            print(f"  Class {cls_idx}: {cls_iou:.4f}")
        print(f"Mean IoU: {miou:.4f}")

        for cls_idx, cls_iou in enumerate(ious):
            logger.info(f"Class {cls_idx:>2} IoU: {cls_iou:.4f}")
        logger.info(f"Mean IoU: {miou:.4f}")

        # Save
        ckpt_name = f"epoch{epoch:02d}_miou{miou:.4f}.pth"
        torch.save(net.module.state_dict(), os.path.join(args.respath, ckpt_name))
        if miou > best_miou:
            best_miou = miou
            torch.save(net.module.state_dict(), os.path.join(args.respath, 'best_mIoU.pth'))

    logger.info("Training complete")

if __name__ == '__main__':
    train()

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


use pretrain model /kaggle/working/STDC-Seg/pretrained_models/STDCNet813M_73.91.tar


Epoch 1/20 Training: 100%|██████████| 193/193 [00:46<00:00,  4.15batch/s, loss=21.8889]


Epoch 1 Training complete. Avg Loss: 27.6975


Epoch 1/20 Validation: 100%|██████████| 166/166 [01:29<00:00,  1.87batch/s, valloss=8.0694] 


...

Epoch 19 Validation complete. Avg Loss: 28.1996
Per-class IoU:
  Class 0: 0.5354
  Class 1: 0.3817
  Class 2: 0.2937
  Class 3: 0.3050
  Class 4: 0.1432
  Class 5: 0.1367
  Class 6: 0.4347
Mean IoU: 0.3186


Epoch 20/20 Training: 100%|██████████| 193/193 [00:39<00:00,  4.92batch/s, loss=10.1442]


Epoch 20 Training complete. Avg Loss: 8.1664


Epoch 20/20 Validation: 100%|██████████| 166/166 [01:03<00:00,  2.63batch/s, valloss=10.6387]


Epoch 20 Validation complete. Avg Loss: 27.4577
Per-class IoU:
  Class 0: 0.5401
  Class 1: 0.4190
  Class 2: 0.3360
  Class 3: 0.3257
  Class 4: 0.1456
  Class 5: 0.1115
  Class 6: 0.4359
Mean IoU: 0.3305
